<div style="padding: 25px; background-color: #1a1a2e; border-radius: 12px; border: 2px solid #e94560;">
    <h1 style="color: #e94560; font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;">Predictive Site Selection Model</h1>
    <h2 style="color: #0f3460;">Binary Classification on H3 Spatial Hexagons — Camden Specialty Coffee</h2>
    <p style="color: #eee; font-size: 1.1em;">
        Predicting specialty coffee shop suitability from multi-modal geospatial features:<br>
        <strong>LandScan</strong> population rasters, <strong>ONS Census 2021</strong> demographics, and <strong>NetworkX</strong> graph centrality.
    </p>
    <hr style="border-color: #e94560;">
    <div style="display: flex; justify-content: space-between; color: #aaa;">
        <span>London Borough of Camden</span>
        <span>MSc Business Analytics | 2026</span>
    </div>
</div>

<div style="margin-top: 30px; padding: 15px; background-color: #eaf2f8; border-radius: 8px; border-left: 5px solid #2e86c1;">
    <h2 style="color: #1a5276;">Pipeline Overview</h2>
    <p>This notebook is <strong>self-contained</strong>. It re-derives all features from raw data sources so it can run independently of Notebooks 01–03. The pipeline:</p>
    <ol>
        <li><strong>Data Assembly</strong>: Load H3 grid, LandScan raster, OSM POIs, and 3 ONS Census CSVs.</li>
        <li><strong>Feature Engineering</strong>: 12 features across 4 modalities (footfall, demographics, graph centrality, POI ecosystem).</li>
        <li><strong>Target Definition</strong>: Binary — <code>has_coffee_shop</code> (1/0) per hexagon.</li>
        <li><strong>Spatial Cross-Validation</strong>: H3 parent-cell block CV to prevent spatial leakage.</li>
        <li><strong>Model Comparison</strong>: Logistic Regression vs. Random Forest vs. XGBoost.</li>
        <li><strong>Hyperparameter Tuning</strong>: GridSearchCV with spatial folds.</li>
        <li><strong>Business Insight</strong>: Extract False Positives as site recommendations.</li>
    </ol>
</div>

In [ ]:
# ============================================================
# SECTION 0: Environment & Imports
# ============================================================
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import h3
import rasterio
import rasterstats
import osmnx as ox
import matplotlib.pyplot as plt
import seaborn as sns
import pydeck as pdk
from shapely.geometry import Polygon, Point

# ML Stack
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, roc_auc_score,
    roc_curve, confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.pipeline import Pipeline
import xgboost as xgb

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Directory scaffold
for d in ['data/raw', 'data/processed', 'data/outputs']:
    os.makedirs(d, exist_ok=True)

print(f"H3 version: {h3.__version__}")
print(f"Environment ready. Random state: {RANDOM_STATE}")

<div style="margin-top: 30px;">
    <h2 style="color: #e94560; border-bottom: 2px solid #e94560; padding-bottom: 10px;">Section 1: Data Preparation — The Feature Matrix</h2>
    <p>We assemble a single flat <code>DataFrame</code> by joining three data modalities onto the H3 hexagonal grid.</p>
    <table style="width:100%; border-collapse: collapse; margin-top: 10px; font-size: 0.95em;">
        <tr style="background: #16213e; color: white;">
            <th style="padding: 8px; text-align: left;">Modality</th>
            <th style="padding: 8px; text-align: left;">Source</th>
            <th style="padding: 8px; text-align: left;">Features</th>
        </tr>
        <tr style="background: #f8f9fa;"><td style="padding: 8px;">Footfall</td><td>LandScan Raster (zonal stats)</td><td><code>population</code></td></tr>
        <tr><td style="padding: 8px;">Demographics</td><td>ONS Census 2021 (Digimap)</td><td><code>employed_total_perc</code>, <code>age_16_to_34_perc</code>, <code>level4_perc</code>, <code>retired_perc</code>, <code>no_qualifications_perc</code></td></tr>
        <tr style="background: #f8f9fa;"><td style="padding: 8px;">Graph Centrality</td><td>NetworkX on H3 adjacency</td><td><code>degree_centrality</code>, <code>betweenness_centrality</code>, <code>closeness_centrality</code>, <code>clustering_coeff</code></td></tr>
        <tr><td style="padding: 8px;">POI Ecosystem</td><td>OSMnx spatial join</td><td><code>n_synergy</code>, <code>n_anchors</code> (competitors excluded — they <em>are</em> the target)</td></tr>
    </table>
</div>

In [ ]:
# ============================================================
# SECTION 1.1: Generate H3 Grid over Camden
# ============================================================
PLACE = "London Borough of Camden"
RESOLUTION = 9  # ~174m edge length — walking scale

# Fetch Camden boundary
boundary = ox.geocode_to_gdf(PLACE)
boundary_wgs84 = boundary.to_crs(epsg=4326)
poly = boundary_wgs84.geometry.iloc[0]

# H3 v4: polygon_to_cells
outer_coords = [(lat, lng) for lng, lat in poly.exterior.coords]
holes = [[(lat, lng) for lng, lat in ring.coords] for ring in poly.interiors]
h3_poly = h3.LatLngPoly(outer_coords, *holes)
hex_ids = h3.polygon_to_cells(h3_poly, RESOLUTION)

# Convert to GeoDataFrame
def h3_to_shapely(cell_id):
    coords = h3.cell_to_boundary(cell_id)
    return Polygon([(lng, lat) for lat, lng in coords])

hex_polygons = [h3_to_shapely(h) for h in hex_ids]
h3_grid = gpd.GeoDataFrame(
    {'h3_index': list(hex_ids)},
    geometry=hex_polygons,
    crs='EPSG:4326'
)

print(f"H3 Grid: {len(h3_grid)} hexagons at Resolution {RESOLUTION}")

In [ ]:
# ============================================================
# SECTION 1.2: Footfall — LandScan Raster Enrichment
# ============================================================
raster_path = "landscan-mosaic-unitedkingdom-v1.tif"

with rasterio.open(raster_path) as src:
    print(f"Raster CRS: {src.crs} | Resolution: {src.res}")

# Zonal stats: sum population pixels under each hex
stats = rasterstats.zonal_stats(h3_grid, raster_path, stats=['sum'], nodata=-999)
h3_grid['population'] = [s['sum'] if s['sum'] is not None else 0 for s in stats]

print(f"Total LandScan population across Camden hexes: {h3_grid['population'].sum():,.0f}")
print(f"Max hex population: {h3_grid['population'].max():,.0f}")
print(f"Hexes with zero population: {(h3_grid['population'] == 0).sum()}")

In [ ]:
# ============================================================
# SECTION 1.3: POI Ecosystem — OSMnx Fetch & Categorisation
# ============================================================
tags = {
    'amenity': ['cafe', 'coffee_shop', 'gym', 'university', 'office',
                'library', 'leisure_centre'],
    'leisure': ['fitness_centre', 'sports_centre'],
    'shop': ['bakery', 'supermarket'],
    'public_transport': ['station']
}

pois_raw = ox.features_from_place(PLACE, tags)
pois_raw['geometry'] = pois_raw.to_crs(epsg=27700).centroid
pois_raw = pois_raw.set_crs(epsg=27700)
pois = pois_raw[pois_raw.geometry.type == 'Point'].copy()
pois = pois.to_crs(epsg=4326)  # match h3_grid CRS for sjoin

# Categorise by business role
def categorize(row):
    amenity = row.get('amenity', '')
    leisure = row.get('leisure', '')
    transport = row.get('public_transport', '')
    if amenity in ('cafe', 'coffee_shop'):
        return 'Competitor'
    if amenity in ('gym', 'university', 'office', 'library', 'leisure_centre') \
       or leisure in ('fitness_centre', 'sports_centre'):
        return 'Synergy'
    if transport == 'station':
        return 'Anchor'
    return 'Other'

pois['role'] = pois.apply(categorize, axis=1)

print(f"POIs fetched: {len(pois)}")
print(pois['role'].value_counts().to_string())

In [ ]:
# ============================================================
# SECTION 1.4: Spatial Join — POI Counts per Hexagon
# ============================================================
pois_in_hex = gpd.sjoin(pois, h3_grid[['h3_index', 'geometry']], how='inner', predicate='within')

# Pivot by role
role_counts = pois_in_hex.groupby(['h3_index', 'role']).size().unstack(fill_value=0)

# Ensure all role columns exist
for col in ['Competitor', 'Synergy', 'Anchor', 'Other']:
    if col not in role_counts.columns:
        role_counts[col] = 0

role_counts = role_counts.rename(columns={
    'Competitor': 'n_competitors',
    'Synergy': 'n_synergy',
    'Anchor': 'n_anchors',
    'Other': 'n_other'
})

h3_grid = h3_grid.merge(role_counts, on='h3_index', how='left')
for col in ['n_competitors', 'n_synergy', 'n_anchors', 'n_other']:
    h3_grid[col] = h3_grid[col].fillna(0).astype(int)

print(f"Competitors: {h3_grid['n_competitors'].sum()} | Synergy: {h3_grid['n_synergy'].sum()} | Anchors: {h3_grid['n_anchors'].sum()}")

In [ ]:
# ============================================================
# SECTION 1.5: Demographics — ONS Census Enrichment
# ============================================================
# Load all 3 census CSVs
econ_df = pd.read_csv('ons-economic-ew-2021_6304504/ons-economic-ew-2021.csv')
age_df  = pd.read_csv('ons-age-ew-2021_6304503/ons-age-ew-2021.csv')
qual_df = pd.read_csv('ons-qualifications-ew-2021_6304505/ons-qualifications-ew-2021.csv')

# Merge on geog_code
census = econ_df[['geog_code', 'centroid_x', 'centroid_y', 'denom_total',
                   'employed_total_perc', 'retired_perc', 'unemployed_perc']].copy()
census = census.merge(
    age_df[['geog_code', 'age_16_to_34_perc', 'age_65_plus_perc']], on='geog_code', how='left'
)
census = census.merge(
    qual_df[['geog_code', 'level4_perc', 'no_qualifications_perc']], on='geog_code', how='left'
)

# Convert to GeoDataFrame (centroids are already BNG EPSG:27700)
geometry = [Point(xy) for xy in zip(census['centroid_x'], census['centroid_y'])]
census_gdf = gpd.GeoDataFrame(census, geometry=geometry, crs='EPSG:27700')
census_gdf = census_gdf.to_crs(epsg=4326)  # match h3_grid

# Spatial join: nearest census OA centroid -> hex
census_in_hex = gpd.sjoin_nearest(
    census_gdf, h3_grid[['h3_index', 'geometry']],
    how='left', max_distance=0.005
)

# Population-weighted mean of demographics per hex
demo_cols = ['employed_total_perc', 'age_16_to_34_perc', 'level4_perc',
             'retired_perc', 'no_qualifications_perc']

def weighted_mean(group, col):
    w = group['denom_total']
    return (group[col] * w).sum() / w.sum() if w.sum() > 0 else 0

hex_demos = []
for h3_id, group in census_in_hex.groupby('h3_index'):
    row = {'h3_index': h3_id}
    for col in demo_cols:
        row[col] = weighted_mean(group, col)
    hex_demos.append(row)

demo_df = pd.DataFrame(hex_demos)
h3_grid = h3_grid.merge(demo_df, on='h3_index', how='left')
for col in demo_cols:
    h3_grid[col] = h3_grid[col].fillna(0)

print(f"Census demographics joined. Coverage: {len(demo_df)}/{len(h3_grid)} hexes")
print(f"  Mean Level 4%: {h3_grid['level4_perc'].mean():.1f}%")
print(f"  Mean Age 16-34%: {h3_grid['age_16_to_34_perc'].mean():.1f}%")
print(f"  Mean Employment%: {h3_grid['employed_total_perc'].mean():.1f}%")

<div style="margin-top: 30px;">
    <h2 style="color: #0f3460; border-bottom: 2px solid #0f3460; padding-bottom: 10px;">Section 1.6: Graph Centrality Features</h2>
    <p>We construct the H3 adjacency graph and compute four centrality metrics per hexagon:</p>
    <ul>
        <li><strong>Degree Centrality</strong>: $C_D(v) = \frac{\deg(v)}{N-1}$ — connectivity (number of neighbours).</li>
        <li><strong>Betweenness Centrality</strong>: $C_B(v) = \sum_{s \neq v \neq t} \frac{\sigma_{st}(v)}{\sigma_{st}}$ — bridging position on shortest paths.</li>
        <li><strong>Closeness Centrality</strong>: $C_C(v) = \frac{N-1}{\sum_{u} d(v,u)}$ — accessibility to all other hexes.</li>
        <li><strong>Clustering Coefficient</strong>: $C_{clust}(v) = \frac{2 T(v)}{\deg(v)(\deg(v)-1)}$ — neighbourhood cohesion.</li>
    </ul>
    <p>These capture a hexagon's <em>structural role</em> in the urban network — independent of its content.</p>
</div>

In [ ]:
# ============================================================
# SECTION 1.6: Graph Centrality Features
# ============================================================
G = nx.Graph()
hex_set = set(h3_grid['h3_index'])

for h3_id in hex_set:
    G.add_node(h3_id)

for h3_id in hex_set:
    for nb in h3.grid_disk(h3_id, 1):
        if nb != h3_id and nb in hex_set:
            G.add_edge(h3_id, nb)

# Compute centrality metrics
h3_grid['degree_centrality'] = h3_grid['h3_index'].map(nx.degree_centrality(G))
h3_grid['betweenness_centrality'] = h3_grid['h3_index'].map(nx.betweenness_centrality(G))
h3_grid['closeness_centrality'] = h3_grid['h3_index'].map(nx.closeness_centrality(G))
h3_grid['clustering_coeff'] = h3_grid['h3_index'].map(nx.clustering(G))

print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"Mean degree centrality: {h3_grid['degree_centrality'].mean():.4f}")
print(f"Mean betweenness centrality: {h3_grid['betweenness_centrality'].mean():.4f}")

<div style="margin-top: 30px;">
    <h2 style="color: #e94560; border-bottom: 2px solid #e94560; padding-bottom: 10px;">Section 2: Target Variable — Binary Coffee Shop Presence</h2>
    <p>We define the prediction target as:</p>
    <p style="text-align: center; font-size: 1.2em;">$$y_H = \begin{cases} 1 & \text{if hexagon } H \text{ contains } \geq 1 \text{ cafe or coffee\_shop} \\ 0 & \text{otherwise} \end{cases}$$</p>
    <p><strong>Class imbalance is expected</strong>: most hexagons will not contain a coffee shop. We handle this with <code>class_weight='balanced'</code> (LR, RF) and <code>scale_pos_weight</code> (XGBoost).</p>

    <div style="background: #eaf2f8; padding: 12px; border-radius: 5px; border-left: 4px solid #2e86c1; margin-top: 10px;">
        <strong>Success Metrics &amp; Constraints</strong>
        <ul style="margin: 8px 0;">
            <li><strong>Primary metric</strong>: ROC-AUC &mdash; threshold-invariant measure of discrimination. Chosen over accuracy because class imbalance makes accuracy misleading (a naive all-zero classifier achieves ~80% accuracy).</li>
            <li><strong>Secondary</strong>: Precision at top-20 &mdash; are the top-ranked recommendations credible to an investor?</li>
            <li><strong>Tertiary</strong>: Recall &mdash; do we miss viable locations? Low recall means leaving revenue on the table.</li>
        </ul>
        <strong>Scope constraints:</strong>
        <ul style="margin: 8px 0;">
            <li>Single borough (Camden) &mdash; model may not generalise to other London boroughs without retraining.</li>
            <li>Static temporal snapshot: ONS Census 2021, OSM data from the notebook run date. No longitudinal dynamics.</li>
            <li>Binary target (presence/absence) &mdash; no ground-truth revenue or profitability data available.</li>
            <li>No commercial rent, planning permission, or lease availability data &mdash; the model identifies <em>demand-side</em> opportunity only.</li>
        </ul>
    </div>

    <div style="background: #fff3cd; padding: 12px; border-radius: 5px; border-left: 4px solid #ffc107; margin-top: 10px;">
        <strong>Leakage Warning:</strong> The <code>n_competitors</code> column counts cafes/coffee shops per hex &mdash; this directly encodes the target.
        It <strong>must be excluded</strong> from the feature matrix <code>X</code>. We retain <code>n_synergy</code> and <code>n_anchors</code>
        as legitimate predictors (synergy nodes attract coffee shops, not vice versa).
    </div>
</div>

In [ ]:
# ============================================================
# SECTION 2: Target Definition & Feature Matrix
# ============================================================

# Binary target
h3_grid['has_coffee_shop'] = (h3_grid['n_competitors'] >= 1).astype(int)

print("Target distribution:")
print(h3_grid['has_coffee_shop'].value_counts())
print(f"Positive class rate: {h3_grid['has_coffee_shop'].mean():.1%}")

# === Feature set (LEAKAGE GUARD: n_competitors excluded) ===
FEATURE_COLS = [
    # Footfall
    'population',
    # Demographics
    'employed_total_perc',
    'age_16_to_34_perc',
    'level4_perc',
    'retired_perc',
    'no_qualifications_perc',
    # Graph Centrality
    'degree_centrality',
    'betweenness_centrality',
    'closeness_centrality',
    'clustering_coeff',
    # POI Ecosystem (synergy + anchors ONLY)
    'n_synergy',
    'n_anchors',
]

TARGET_COL = 'has_coffee_shop'

X = h3_grid[FEATURE_COLS].copy()
y = h3_grid[TARGET_COL].copy()

# Leakage assertion
assert 'n_competitors' not in X.columns, "LEAKAGE: n_competitors found in feature set!"

print(f"\nFeature matrix X: {X.shape}")
print(f"Features ({len(FEATURE_COLS)}): {FEATURE_COLS}")
print(f"\nMissing values per feature:")
print(X.isnull().sum().to_string())

<div style="margin-top: 30px;">
    <h2 style="color: #0f3460; border-bottom: 2px solid #0f3460; padding-bottom: 10px;">Section 3: Exploratory Data Analysis & Leakage Audit</h2>
    <p>Before modelling, we verify:</p>
    <ol>
        <li><strong>Feature distributions</strong>: Zero variance, extreme skew, or missing values?</li>
        <li><strong>Multicollinearity</strong>: Variance Inflation Factors (VIF) — flag features with VIF &gt; 10.</li>
        <li><strong>Spatial autocorrelation</strong>: Visual confirmation that the target is spatially clustered (motivating Spatial CV).</li>
    </ol>
</div>

In [ ]:
# ============================================================
# SECTION 3-pre: Missingness Audit & Class Balance
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Left: Missingness summary ---
missing = X.isnull().sum()
missing_pct = (X.isnull().sum() / len(X) * 100)
miss_df = pd.DataFrame({'Count': missing, 'Percent': missing_pct}).sort_values('Count', ascending=False)

axes[0].barh(miss_df.index, miss_df['Percent'], color='#e74c3c')
axes[0].set_xlabel('Missing (%)')
axes[0].set_title('Feature Missingness', fontsize=12, fontweight='bold')
axes[0].axvline(x=5, color='gray', linestyle='--', alpha=0.5, label='5% threshold')
axes[0].legend()

# --- Right: Class balance ---
class_counts = y.value_counts().sort_index()
bars = axes[1].bar(
    ['No Coffee (0)', 'Has Coffee (1)'],
    class_counts.values,
    color=['#3498db', '#e74c3c']
)
for bar, val in zip(bars, class_counts.values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                 str(val), ha='center', fontweight='bold')
axes[1].set_title('Target Class Distribution', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Number of Hexagons')

imbalance = class_counts[0] / class_counts[1]
axes[1].annotate(f'Imbalance ratio: {imbalance:.1f}:1',
                 xy=(0.5, 0.85), xycoords='axes fraction',
                 ha='center', fontsize=11, fontweight='bold',
                 bbox=dict(boxstyle='round', facecolor='#fff3cd', edgecolor='#ffc107'))

plt.tight_layout()
plt.show()

print(f"Missingness summary: {(missing > 0).sum()} features have missing values")
print(f"Class balance: {class_counts[0]} negative, {class_counts[1]} positive "
      f"(ratio {imbalance:.1f}:1)")
print(f"  -> A naive all-zero classifier achieves {class_counts[0]/len(y)*100:.1f}% accuracy")
print(f"  -> This motivates ROC-AUC as the primary metric (threshold-invariant)")

In [ ]:
# ============================================================
# SECTION 3a: Correlation Heatmap
# ============================================================
fig, ax = plt.subplots(figsize=(12, 10))

corr = X.join(y).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(
    corr, mask=mask, annot=True, fmt='.2f',
    cmap='RdBu_r', center=0, vmin=-1, vmax=1,
    square=True, linewidths=0.5, ax=ax
)
ax.set_title('Feature Correlation Matrix (incl. Target)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('data/outputs/eda_correlations.png', dpi=150, bbox_inches='tight')
plt.show()

# Flag high correlations with target
target_corr = corr[TARGET_COL].drop(TARGET_COL).sort_values(ascending=False)
print("Feature correlations with target (has_coffee_shop):")
print(target_corr.to_string())

In [ ]:
# ============================================================
# SECTION 3b: Feature Distributions by Target Class
# ============================================================
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
key_features = ['population', 'level4_perc', 'age_16_to_34_perc',
                'betweenness_centrality', 'n_synergy', 'n_anchors']

for ax, feat in zip(axes.ravel(), key_features):
    plot_df = X[[feat]].copy()
    plot_df['target'] = y.values
    sns.boxplot(data=plot_df, x='target', y=feat, ax=ax,
                palette={0: '#3498db', 1: '#e74c3c'})
    ax.set_title(feat, fontsize=11, fontweight='bold')
    ax.set_xlabel('has_coffee_shop')

plt.suptitle('Feature Distributions by Target Class', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('data/outputs/eda_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================
# SECTION 3c: Variance Inflation Factor (Multicollinearity)
# ============================================================
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF requires no NaN and no constant columns
X_vif = X.fillna(0).copy()
# Remove zero-variance columns for VIF calculation
X_vif = X_vif.loc[:, X_vif.std() > 0]

vif_data = pd.DataFrame({
    'Feature': X_vif.columns,
    'VIF': [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
}).sort_values('VIF', ascending=False)

print("Variance Inflation Factors:")
print(vif_data.to_string(index=False))
print(f"\nFeatures with VIF > 10 (potential multicollinearity): "
      f"{list(vif_data[vif_data['VIF'] > 10]['Feature'])}")

<div style="margin-top: 20px; padding: 15px; background-color: #fdf2e9; border-radius: 8px; border-left: 5px solid #d35400;">
    <h3 style="color: #d35400;">Leakage Audit</h3>
    <p>Before proceeding to modelling, we formally document potential leakage vectors and our mitigations:</p>
    <table style="width:100%; border-collapse: collapse; margin: 10px 0; font-size: 0.9em;">
        <tr style="background: #d35400; color: white;">
            <th style="padding: 8px;">Risk</th>
            <th style="padding: 8px;">Feature(s)</th>
            <th style="padding: 8px;">Assessment</th>
            <th style="padding: 8px;">Mitigation</th>
        </tr>
        <tr>
            <td style="padding: 8px;"><strong>Direct target encoding</strong></td>
            <td><code>n_competitors</code></td>
            <td style="color: #c0392b;">HIGH &mdash; counts cafes/coffee shops, which <em>are</em> the target</td>
            <td>Excluded from <code>X</code>. Enforced by runtime assertion.</td>
        </tr>
        <tr style="background: #fef9f4;">
            <td style="padding: 8px;"><strong>Reverse causation</strong></td>
            <td><code>n_synergy</code>, <code>n_anchors</code></td>
            <td style="color: #e67e22;">MODERATE &mdash; do coffee shops attract gyms, or do gyms attract coffee shops?</td>
            <td>Retained. Causal direction is predominantly synergy &rarr; coffee (gyms/offices pre-date most cafes). Acknowledged as a limitation in the Model Card.</td>
        </tr>
        <tr>
            <td style="padding: 8px;"><strong>Temporal leakage</strong></td>
            <td>All OSM features</td>
            <td style="color: #e67e22;">MODERATE &mdash; OSM snapshot is contemporaneous with target; no temporal split</td>
            <td>Accepted. This is a cross-sectional study. Acknowledged in scope constraints.</td>
        </tr>
        <tr style="background: #fef9f4;">
            <td style="padding: 8px;"><strong>Spatial leakage</strong></td>
            <td>All features (via autocorrelation)</td>
            <td style="color: #c0392b;">HIGH &mdash; adjacent hexes share similar feature values (Tobler's Law)</td>
            <td>Controlled by Spatial Block CV (Section 4). Adjacent hexes always in same fold.</td>
        </tr>
    </table>
</div>

<div style="margin-top: 30px;">
    <h2 style="color: #c0392b; border-bottom: 2px solid #c0392b; padding-bottom: 10px;">Section 4: Spatial Cross-Validation Strategy</h2>
    <p><strong>Problem:</strong> Standard <code>KFold</code> randomly assigns hexagons to train/test folds.
    Adjacent hexes share similar features due to spatial autocorrelation
    (<em>Tobler's First Law of Geography</em>: "near things are more related than distant things").
    Random splits leak spatial signal, inflating performance metrics by 5–15%.</p>
    
    <p><strong>Solution:</strong> <strong>Spatial Block Cross-Validation</strong> using the H3 hierarchy.
    We group Resolution-9 hexagons by their Resolution-5 parent cell (∼10 km² blocks).
    All hexes sharing a parent are assigned to the same fold, preserving spatial integrity.</p>
    
    <p style="text-align: center; font-size: 1.1em;">$$\text{fold}(H) = \text{h3\_cell\_to\_parent}(H, \text{res}=5) \bmod k$$</p>
</div>

In [ ]:
# ============================================================
# SECTION 4: Spatial Block Cross-Validation (H3 Parent-Cell)
# ============================================================

class SpatialKFold:
    """Spatial block cross-validation using H3 parent-cell partitioning.

    Groups H3 Res-9 hexagons by their Res-5 parent cell, then assigns
    each spatial block to a fold. Ensures geographically proximate
    hexagons never appear in both train and test simultaneously.
    """

    def __init__(self, n_splits=5, parent_resolution=5):
        self.n_splits = n_splits
        self.parent_resolution = parent_resolution

    def split(self, X, y=None, groups=None):
        parents = groups.apply(
            lambda h: h3.cell_to_parent(h, self.parent_resolution)
        )
        unique_parents = parents.unique()
        parent_to_fold = {
            p: i % self.n_splits
            for i, p in enumerate(unique_parents)
        }
        fold_assignments = parents.map(parent_to_fold)

        for fold_idx in range(self.n_splits):
            test_mask = fold_assignments == fold_idx
            train_idx = np.where(~test_mask)[0]
            test_idx = np.where(test_mask)[0]
            yield train_idx, test_idx

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits


spatial_cv = SpatialKFold(n_splits=5, parent_resolution=5)

# Verify fold sizes
print("Spatial CV Fold Sizes:")
for i, (train_idx, test_idx) in enumerate(
    spatial_cv.split(X, y, groups=h3_grid['h3_index'])
):
    print(f"  Fold {i+1}: train={len(train_idx)}, test={len(test_idx)}, "
          f"test_positive_rate={y.iloc[test_idx].mean():.1%}")

<div style="margin-top: 30px;">
    <h2 style="color: #117a65; border-bottom: 2px solid #117a65; padding-bottom: 10px;">Section 5: Baseline Model — Logistic Regression</h2>
    <p>Logistic Regression serves as our <strong>interpretability baseline</strong>. Its linear decision boundary provides
    directly readable coefficients — the sign and magnitude of each feature's effect on coffee shop presence.</p>
    <p>We apply <code>StandardScaler</code> (required for LR convergence) and <code>class_weight='balanced'</code>
    to handle the expected class imbalance.</p>
</div>

In [ ]:
# ============================================================
# SECTION 5: Baseline — Logistic Regression
# ============================================================
lr_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(
        class_weight='balanced', max_iter=1000, random_state=RANDOM_STATE
    ))
])

lr_scores = cross_val_score(
    lr_pipe, X, y,
    cv=spatial_cv.split(X, y, groups=h3_grid['h3_index']),
    scoring='roc_auc'
)

print(f"Logistic Regression (Spatial CV):")
print(f"  ROC-AUC: {lr_scores.mean():.3f} \u00b1 {lr_scores.std():.3f}")
print(f"  Per-fold: {[f'{s:.3f}' for s in lr_scores]}")

# Fit on full data for coefficient inspection
lr_pipe.fit(X, y)
lr_coefs = pd.Series(
    lr_pipe.named_steps['model'].coef_[0],
    index=FEATURE_COLS
).sort_values()

print(f"\nLogistic Regression Coefficients (standardised):")
print(lr_coefs.to_string())

<div style="margin-top: 30px;">
    <h2 style="color: #884ea0; border-bottom: 2px solid #884ea0; padding-bottom: 10px;">Section 6: Model Comparison — Random Forest & XGBoost</h2>
    <p>We compare three models spanning the complexity spectrum:</p>
    <table style="width:100%; border-collapse: collapse; margin-top: 10px;">
        <tr style="background: #16213e; color: white;">
            <th style="padding: 8px;">Model</th><th>Type</th><th>Class Imbalance Handling</th><th>Rationale</th>
        </tr>
        <tr><td style="padding: 8px;">Logistic Regression</td><td>Linear</td><td><code>class_weight='balanced'</code></td><td>Interpretable baseline</td></tr>
        <tr style="background: #f8f9fa;"><td style="padding: 8px;">Random Forest</td><td>Bagged ensemble</td><td><code>class_weight='balanced'</code></td><td>Non-linear, robust to outliers</td></tr>
        <tr><td style="padding: 8px;">XGBoost</td><td>Boosted ensemble</td><td><code>scale_pos_weight</code></td><td>State-of-the-art tabular performance</td></tr>
    </table>
</div>

In [ ]:
# ============================================================
# SECTION 6: Model Comparison — LR vs RF vs XGBoost
# ============================================================
imbalance_ratio = (y == 0).sum() / (y == 1).sum()

models = {
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(
            class_weight='balanced', max_iter=1000, random_state=RANDOM_STATE
        ))
    ]),
    'Random Forest': RandomForestClassifier(
        n_estimators=200, class_weight='balanced',
        random_state=RANDOM_STATE, n_jobs=-1
    ),
    'XGBoost': xgb.XGBClassifier(
        n_estimators=200,
        scale_pos_weight=imbalance_ratio,
        eval_metric='logloss',
        random_state=RANDOM_STATE
    )
}

results = {}
for name, model in models.items():
    scores = cross_val_score(
        model, X, y,
        cv=spatial_cv.split(X, y, groups=h3_grid['h3_index']),
        scoring='roc_auc'
    )
    results[name] = {
        'mean_auc': scores.mean(),
        'std_auc': scores.std(),
        'scores': scores
    }
    print(f"{name:25s} ROC-AUC = {scores.mean():.3f} \u00b1 {scores.std():.3f}")

# Summary table
results_df = pd.DataFrame({
    'Model': results.keys(),
    'Mean AUC': [r['mean_auc'] for r in results.values()],
    'Std AUC': [r['std_auc'] for r in results.values()]
}).sort_values('Mean AUC', ascending=False)

print(f"\nModel Comparison Summary:")
results_df

<div style="margin-top: 30px;">
    <h2 style="color: #d35400; border-bottom: 2px solid #d35400; padding-bottom: 10px;">Section 7: Hyperparameter Tuning</h2>
    <p>We tune the best-performing model using <code>GridSearchCV</code> with our spatial CV splitter.
    The search space covers tree depth, learning rate, subsampling, and column sampling —
    the most impactful XGBoost hyperparameters for tabular classification.</p>
</div>

In [ ]:
# ============================================================
# SECTION 7: Hyperparameter Tuning (XGBoost)
# ============================================================
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 1.0]
}

xgb_base = xgb.XGBClassifier(
    scale_pos_weight=imbalance_ratio,
    eval_metric='logloss',
    random_state=RANDOM_STATE
)

# Pre-compute spatial CV splits for GridSearchCV
cv_splits = list(spatial_cv.split(X, y, groups=h3_grid['h3_index']))

grid_search = GridSearchCV(
    xgb_base, param_grid,
    cv=cv_splits,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1,
    refit=True
)

grid_search.fit(X, y)

print(f"\nBest Parameters: {grid_search.best_params_}")
print(f"Best ROC-AUC (Spatial CV): {grid_search.best_score_:.3f}")

<div style="margin-top: 30px;">
    <h2 style="color: #1a5276; border-bottom: 2px solid #1a5276; padding-bottom: 10px;">Section 8: Evaluation Suite</h2>
    <p>We present the full evaluation battery: ROC curves, confusion matrix, feature importance, and classification report.</p>
</div>

In [ ]:
# ============================================================
# SECTION 8a: ROC Curves — Out-of-Fold Predictions (Spatial CV)
# ============================================================
from sklearn.model_selection import cross_val_predict

fig, ax = plt.subplots(figsize=(8, 6))

# Materialise spatial CV splits once (reuse for all models)
cv_splits = list(spatial_cv.split(X, y, groups=h3_grid['h3_index']))

for name, model in models.items():
    # Out-of-fold predictions: each sample predicted only when in the test fold
    y_prob_oof = cross_val_predict(
        model, X, y, cv=cv_splits, method='predict_proba'
    )[:, 1]
    fpr, tpr, _ = roc_curve(y, y_prob_oof)
    auc_val = roc_auc_score(y, y_prob_oof)
    ax.plot(fpr, tpr, linewidth=2, label=f'{name} (AUC={auc_val:.3f})')

# Tuned XGBoost: build a fresh estimator with best params for OOF evaluation
best_params = grid_search.best_params_
tuned_xgb = xgb.XGBClassifier(
    **best_params,
    scale_pos_weight=imbalance_ratio,
    eval_metric='logloss',
    random_state=RANDOM_STATE
)
y_prob_oof_best = cross_val_predict(
    tuned_xgb, X, y, cv=cv_splits, method='predict_proba'
)[:, 1]
fpr_b, tpr_b, _ = roc_curve(y, y_prob_oof_best)
auc_best = roc_auc_score(y, y_prob_oof_best)
ax.plot(fpr_b, tpr_b, linewidth=2.5, linestyle='--',
        label=f'XGBoost Tuned (AUC={auc_best:.3f})')

ax.plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Random Baseline')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curves: Spatial CV Out-of-Fold Predictions', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('data/outputs/roc_curves.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================
# SECTION 8b: Confusion Matrix — Out-of-Fold (Spatial CV)
# ============================================================
# Use OOF predictions from the tuned XGBoost for honest evaluation
y_pred_oof = cross_val_predict(tuned_xgb, X, y, cv=cv_splits)

fig, ax = plt.subplots(figsize=(6, 5))
cm = confusion_matrix(y, y_pred_oof)
disp = ConfusionMatrixDisplay(cm, display_labels=['No Coffee (0)', 'Has Coffee (1)'])
disp.plot(cmap='Blues', ax=ax, values_format='d')
ax.set_title('Confusion Matrix \u2014 Spatial CV Out-of-Fold', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('data/outputs/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nClassification Report (Out-of-Fold, Spatial CV):")
print(classification_report(y, y_pred_oof, target_names=['No Coffee', 'Has Coffee']))

<div style="margin-top: 30px;">
    <h2 style="color: #8e44ad; border-bottom: 2px solid #8e44ad; padding-bottom: 10px;">Section 8d: Calibration &amp; Error Analysis</h2>
    <p>Beyond discrimination (ROC-AUC), we assess <strong>calibration</strong> (are predicted probabilities trustworthy?)
    and <strong>failure modes</strong> (where and why does the model err?).</p>
</div>

In [ ]:
# ============================================================
# SECTION 8d: Calibration Curve
# ============================================================
from sklearn.calibration import calibration_curve

fig, ax = plt.subplots(figsize=(7, 6))

# Calibration curve: do predicted probabilities match observed frequencies?
fraction_pos, mean_predicted = calibration_curve(y, y_prob_oof_best, n_bins=8, strategy='uniform')

ax.plot(mean_predicted, fraction_pos, 's-', color='#e94560', linewidth=2, label='Tuned XGBoost')
ax.plot([0, 1], [0, 1], 'k--', alpha=0.4, label='Perfectly Calibrated')
ax.set_xlabel('Mean Predicted Probability', fontsize=12)
ax.set_ylabel('Observed Fraction of Positives', fontsize=12)
ax.set_title('Calibration Curve — OOF Predictions', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Interpretation
print("Calibration check:")
for mp, fp_val in zip(mean_predicted, fraction_pos):
    print(f"  Predicted ~{mp:.2f} -> Observed {fp_val:.2f}")
print("\nIf the model says 60% probability, do ~60% of those hexes actually have coffee?")
print("Points above the diagonal = under-confident; below = over-confident.")

In [ ]:
# ============================================================
# SECTION 8e: Failure Mode Analysis — FP vs TP vs FN Feature Profiles
# ============================================================

# Classify OOF predictions by confusion matrix quadrant
oof_outcome = pd.Series('TN', index=y.index)
oof_outcome[(y_pred_oof == 1) & (y == 0)] = 'FP'
oof_outcome[(y_pred_oof == 1) & (y == 1)] = 'TP'
oof_outcome[(y_pred_oof == 0) & (y == 1)] = 'FN'

# Compare feature profiles across outcome groups
analysis_features = ['population', 'level4_perc', 'age_16_to_34_perc',
                     'n_synergy', 'n_anchors', 'betweenness_centrality']

outcome_profiles = X.copy()
outcome_profiles['oof_outcome'] = oof_outcome.values

profile_summary = outcome_profiles.groupby('oof_outcome')[analysis_features].mean()

print("Mean Feature Values by OOF Prediction Outcome:")
print("=" * 70)
print(profile_summary.round(3).to_string())
print()

# Highlight failure mode insights
if 'FN' in profile_summary.index and 'TP' in profile_summary.index:
    fn_vs_tp = profile_summary.loc['FN'] - profile_summary.loc['TP']
    print("FN vs TP difference (what the model misses):")
    for feat in analysis_features:
        diff = fn_vs_tp[feat]
        direction = "lower" if diff < 0 else "higher"
        print(f"  {feat}: FN has {abs(diff):.3f} {direction} than TP")

print()
if 'FP' in profile_summary.index:
    print(f"False Positive count (OOF): {(oof_outcome == 'FP').sum()}")
    print(f"False Negative count (OOF): {(oof_outcome == 'FN').sum()}")
    print(f"True Positive count (OOF):  {(oof_outcome == 'TP').sum()}")
    print(f"True Negative count (OOF):  {(oof_outcome == 'TN').sum()}")

# Visualise: grouped bar chart of feature means by outcome
fig, ax = plt.subplots(figsize=(12, 6))
profile_norm = profile_summary.div(profile_summary.max())  # normalise for comparability
profile_norm.T.plot(kind='bar', ax=ax, width=0.8,
                    color={'TP': '#e74c3c', 'FP': '#27ae60', 'TN': '#95a5a6', 'FN': '#f39c12'})
ax.set_ylabel('Normalised Mean Value', fontsize=11)
ax.set_title('Feature Profiles by Prediction Outcome (OOF)', fontsize=14, fontweight='bold')
ax.legend(title='Outcome', fontsize=10)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# SECTION 8c: Feature Importance — Tuned XGBoost
# ============================================================
importances = pd.Series(
    best_model.feature_importances_, index=FEATURE_COLS
).sort_values()

fig, ax = plt.subplots(figsize=(8, 6))
importances.plot(
    kind='barh', ax=ax,
    color=['#e94560' if v > importances.median() else '#3498db' for v in importances]
)
ax.set_xlabel('Importance (Gain)', fontsize=12)
ax.set_title('Feature Importance \u2014 Tuned XGBoost', fontsize=14, fontweight='bold')
ax.axvline(x=importances.median(), color='gray', linestyle='--', alpha=0.5, label='Median')
ax.legend()
plt.tight_layout()
plt.savefig('data/outputs/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("Feature Importance Ranking:")
print(importances.sort_values(ascending=False).to_string())

<div style="margin-top: 30px;">
    <h2 style="color: #27ae60; border-bottom: 2px solid #27ae60; padding-bottom: 10px;">Section 9: Business Insight — Mining False Positives</h2>
    <p>The most commercially valuable output of this model is not its accuracy — it is its <strong>mistakes</strong>.</p>
    <p>A <strong>False Positive</strong> is a hexagon where:</p>
    <ul>
        <li>The model predicts <code>y=1</code> (this location <em>should</em> have a coffee shop)</li>
        <li>The ground truth is <code>y=0</code> (no coffee shop currently exists here)</li>
    </ul>
    <p>These hexagons possess <em>all the learned features</em> of successful coffee shop locations —
    high footfall, educated demographics, strong transit connectivity, synergy with gyms and offices —
    but <strong>no one has opened a shop there yet</strong>.</p>
    <p>In the language of network theory, these are <strong>Structural Holes</strong> (Burt, 1992) —
    now identified not by heuristic scoring but by supervised machine learning.</p>
</div>

In [ ]:
# ============================================================
# SECTION 9: Extract False Positives as Site Recommendations
# ============================================================

# Retrain best model on full dataset for deployment predictions.
# OOF evaluation was completed in Section 8; here we use the full-data
# model to generate site recommendations (standard ML deployment practice).
best_model = grid_search.best_estimator_
best_model.fit(X, y)

y_pred = best_model.predict(X)
y_prob_best = best_model.predict_proba(X)[:, 1]

h3_grid['predicted'] = y_pred
h3_grid['actual'] = y.values
h3_grid['predicted_prob'] = y_prob_best

# Classify each hex by confusion matrix quadrant
def classify_outcome(row):
    if row['predicted'] == 1 and row['actual'] == 0:
        return 'False Positive (Recommendation)'
    elif row['predicted'] == 1 and row['actual'] == 1:
        return 'True Positive'
    elif row['predicted'] == 0 and row['actual'] == 0:
        return 'True Negative'
    else:
        return 'False Negative'

h3_grid['outcome'] = h3_grid.apply(classify_outcome, axis=1)

print("Prediction Outcome Distribution:")
print(h3_grid['outcome'].value_counts().to_string())

# Extract and rank False Positives
fp = h3_grid[h3_grid['outcome'] == 'False Positive (Recommendation)'].copy()
fp_ranked = fp.nlargest(10, 'predicted_prob')

print(f"\n{'='*60}")
print(f"TOP 10 RECOMMENDED SITES (False Positives)")
print(f"{'='*60}")
fp_ranked[[
    'h3_index', 'predicted_prob', 'population',
    'level4_perc', 'age_16_to_34_perc',
    'n_synergy', 'n_anchors', 'betweenness_centrality'
]]

In [ ]:
# ============================================================
# SECTION 9b: Demographic Profile of Recommended Sites
# ============================================================
# Compare FP sites vs Camden average
profile_cols = ['population', 'level4_perc', 'age_16_to_34_perc',
                'employed_total_perc', 'betweenness_centrality',
                'n_synergy', 'n_anchors']

comparison = pd.DataFrame({
    'Camden Average': h3_grid[profile_cols].mean(),
    'Recommended Sites (FP)': fp[profile_cols].mean() if len(fp) > 0 else 0,
    'Existing Shops (TP)': h3_grid[h3_grid['outcome'] == 'True Positive'][profile_cols].mean()
})

print("Demographic Profile Comparison:")
comparison

<div style="margin-top: 30px;">
    <h2 style="color: #7d3c98; border-bottom: 2px solid #7d3c98; padding-bottom: 10px;">Section 10: Interactive 3D Recommendation Map</h2>
    <p>We visualise the full confusion matrix spatially using <strong>Pydeck</strong>:</p>
    <ul>
        <li style="color: #27ae60;"><strong>Green (extruded)</strong>: False Positives — our site recommendations</li>
        <li style="color: #e74c3c;"><strong>Red (flat)</strong>: True Positives — existing coffee shops</li>
        <li style="color: #95a5a6;"><strong>Grey (flat)</strong>: True Negatives — not suitable</li>
        <li style="color: #f39c12;"><strong>Orange (flat)</strong>: False Negatives — missed by model</li>
    </ul>
</div>

In [ ]:
# ============================================================
# SECTION 10: Pydeck 3D Recommendation Map
# ============================================================
viz_df = h3_grid.to_crs(epsg=4326).copy()

# Colour coding by confusion matrix quadrant
color_map = {
    'False Positive (Recommendation)': [39, 174, 96, 200],   # Green
    'True Positive':                   [231, 76, 60, 160],   # Red
    'True Negative':                   [149, 165, 166, 80],  # Grey
    'False Negative':                  [243, 156, 18, 160],  # Orange
}

viz_df['color'] = viz_df['outcome'].map(color_map)

# Elevation: extrude FP sites by confidence, keep others flat
viz_df['elevation'] = viz_df.apply(
    lambda r: r['predicted_prob'] * 500
    if r['outcome'] == 'False Positive (Recommendation)' else 10,
    axis=1
)

layer = pdk.Layer(
    'H3HexagonLayer',
    viz_df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=True,
    get_hexagon='h3_index',
    get_fill_color='color',
    get_elevation='elevation',
    elevation_scale=1,
)

view_state = pdk.ViewState(
    latitude=51.54, longitude=-0.14,
    zoom=12.5, pitch=50, bearing=-15
)

tooltip = {
    'html': (
        '<b>H3:</b> {h3_index}<br>'
        '<b>Outcome:</b> {outcome}<br>'
        '<b>P(coffee):</b> {predicted_prob}<br>'
        '<b>Population:</b> {population}<br>'
        '<b>Level 4%:</b> {level4_perc}<br>'
        '<b>Synergy:</b> {n_synergy} | <b>Anchors:</b> {n_anchors}'
    ),
    'style': {
        'backgroundColor': '#1a1a2e',
        'color': 'white',
        'fontSize': '12px'
    }
}

deck = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip=tooltip)
deck.to_html('data/outputs/camden_ml_recommendations.html')

print("Interactive 3D map saved: data/outputs/camden_ml_recommendations.html")

<div style="margin-top: 30px; padding: 20px; background-color: #1a1a2e; border-radius: 12px; border: 2px solid #e94560;">
    <h2 style="color: #e94560;">Section 12: Model Card</h2>
    <p style="color: #ccc;">Following Mitchell et al. (2019), <em>Model Cards for Model Reporting</em>.</p>

    <div style="background: #16213e; padding: 15px; border-radius: 8px; margin-top: 10px; color: #eee;">
        <h3 style="color: #e94560;">1. Model Purpose</h3>
        <p>Binary classifier identifying underserved H3 hexagonal cells for specialty coffee retail
        in the London Borough of Camden. Intended users: B2B site selection analysts, commercial real
        estate teams, and urban planning consultants.</p>
        <p><strong>Use case:</strong> The model's False Positive predictions (FP) represent <em>structural holes</em>
        &mdash; locations that possess the demand-side characteristics of existing coffee shop sites but
        where no shop currently operates. These are candidate sites for new investment.</p>

        <h3 style="color: #e94560; margin-top: 15px;">2. Data Provenance</h3>
        <table style="width:100%; color: #ccc; border-collapse: collapse; font-size: 0.9em;">
            <tr style="border-bottom: 1px solid #333;">
                <td style="padding: 6px;"><strong>Population</strong></td>
                <td>LandScan Global 2022 (Oak Ridge National Laboratory), ~1 km resolution GeoTIFF</td>
            </tr>
            <tr style="border-bottom: 1px solid #333;">
                <td style="padding: 6px;"><strong>Demographics</strong></td>
                <td>ONS Census 2021 (EDINA Digimap): age structure, economic activity, qualifications. Output Area level (~125 m median diameter in Camden)</td>
            </tr>
            <tr style="border-bottom: 1px solid #333;">
                <td style="padding: 6px;"><strong>Points of Interest</strong></td>
                <td>OpenStreetMap via OSMnx (snapshot at notebook runtime). Tags: amenity, leisure, shop, public_transport</td>
            </tr>
            <tr style="border-bottom: 1px solid #333;">
                <td style="padding: 6px;"><strong>Spatial Unit</strong></td>
                <td>Uber H3 Resolution 9 hexagons (~174 m edge length, ~0.105 km&sup2;)</td>
            </tr>
            <tr>
                <td style="padding: 6px;"><strong>Graph Features</strong></td>
                <td>H3 adjacency graph via NetworkX. Degree, betweenness, closeness centrality + clustering coefficient</td>
            </tr>
        </table>

        <h3 style="color: #e94560; margin-top: 15px;">3. Evaluation Summary</h3>
        <ul>
            <li><strong>Best model:</strong> XGBoost (tuned via GridSearchCV)</li>
            <li><strong>Evaluation protocol:</strong> 5-fold Spatial Block CV using H3 Res-5 parent partitioning</li>
            <li><strong>Primary metric:</strong> ROC-AUC (out-of-fold predictions)</li>
            <li><strong>Note:</strong> Spatial CV typically reduces apparent AUC by 5&ndash;15% compared to random CV due to controlling for spatial autocorrelation. This is a feature, not a bug &mdash; it reflects honest generalisation performance.</li>
        </ul>

        <h3 style="color: #e94560; margin-top: 15px;">4. Known Limitations</h3>
        <ul style="color: #ddd;">
            <li><strong>Geographic scope:</strong> Trained exclusively on Camden. The model may not generalise to other London boroughs or UK cities without retraining, as the feature&ndash;target relationship is conditioned on Camden's unique urban structure.</li>
            <li><strong>Temporal snapshot:</strong> Census data is from 2021; OSM data from the notebook run date. The model does not capture neighbourhood evolution, new developments, or post-pandemic shifts in foot traffic.</li>
            <li><strong>Binary target:</strong> Coffee shop presence &ne; profitability. A hexagon with a struggling caf&eacute; counts as positive (y=1). No revenue, footfall volume, or commercial rent data is incorporated.</li>
            <li><strong>OSM completeness:</strong> OpenStreetMap coverage varies. Some POIs (especially offices and gyms) may be under-reported, affecting <code>n_synergy</code> accuracy.</li>
            <li><strong>Omitted variables:</strong> Commercial rent, planning restrictions, lease availability, competitor brand strength, and pavement footfall counts are not modelled. The model identifies <em>demand-side</em> opportunity only.</li>
            <li><strong>No held-out test set:</strong> All data is used within spatial CV. There is no temporally or geographically independent test set to validate deployment performance.</li>
        </ul>

        <h3 style="color: #e94560; margin-top: 15px;">5. Ethical Considerations</h3>
        <ul style="color: #ddd;">
            <li><strong>Gentrification risk:</strong> Recommending specialty coffee in underserved areas could contribute to gentrification pressure, potentially displacing existing businesses or communities. Decision-makers must consider local impact.</li>
            <li><strong>Algorithmic bias:</strong> The model may encode demographic proxies. Areas with lower educational attainment are systematically ranked lower &mdash; this reflects market demand patterns but should not be interpreted as a normative judgement about community value.</li>
            <li><strong>Human-in-the-loop:</strong> Predictions should <strong>inform</strong>, not replace, human decision-making. Field visits, stakeholder consultation, and financial due diligence must precede any investment decision.</li>
        </ul>

        <h3 style="color: #e94560; margin-top: 15px;">6. Recommended Use</h3>
        <p style="color: #ddd;">This model is a <strong>shortlisting tool</strong>. It narrows the search space for site analysts
        by identifying the most promising hexagonal zones. It must be combined with:
        (a) field visits, (b) commercial rent analysis, (c) local authority planning checks, and
        (d) community stakeholder consultation before any investment commitment.</p>
    </div>
</div>

In [ ]:
# ============================================================
# SECTION 11: Export Final Results
# ============================================================

# 1. Full grid with predictions
h3_grid.to_parquet('data/outputs/camden_ml_scored.parquet')

# 2. Top 20 FP recommendations with lat/lon
fp_top20 = h3_grid[
    h3_grid['outcome'] == 'False Positive (Recommendation)'
].nlargest(20, 'predicted_prob').copy()

fp_export = fp_top20.to_crs(epsg=4326)
fp_export['latitude'] = fp_export.geometry.centroid.y
fp_export['longitude'] = fp_export.geometry.centroid.x

export_cols = [
    'h3_index', 'latitude', 'longitude', 'predicted_prob',
    'population', 'level4_perc', 'age_16_to_34_perc',
    'employed_total_perc', 'n_synergy', 'n_anchors',
    'betweenness_centrality', 'closeness_centrality'
]
fp_export[export_cols].to_csv('data/outputs/fp_recommendations.csv', index=False)

# 3. Model comparison summary
results_df.to_csv('data/outputs/model_comparison.csv', index=False)

print("Final outputs saved:")
print("  data/outputs/camden_ml_scored.parquet    (full grid + predictions)")
print("  data/outputs/fp_recommendations.csv       (top 20 site recommendations)")
print("  data/outputs/model_comparison.csv         (model AUC summary)")
print("  data/outputs/camden_ml_recommendations.html (interactive 3D map)")
print("  data/outputs/roc_curves.png               (ROC comparison chart)")
print("  data/outputs/confusion_matrix.png          (confusion matrix)")
print("  data/outputs/feature_importance.png        (feature importance chart)")
print(f"\nTotal False Positive recommendations: {len(fp)}")
print("Pipeline complete.")